# Análisis RFM

El análisis RFM es un modelo que permite analizar el comportamiento de los clientes, creando segmentos internamente homogéneos y mutuamente heterogéneos a partir de su comportamiento observado en los tickets de transacciones.

El análisis RFM implica el uso de tres variables: 
-	Recencia (actualidad): Tiempo que ha transcurrido desde la última compra. Generalmente se considera que esta es la variable más importante para la segmentación.
-	Frecuencia: Número de compras en un periodo específico de tiempo
-	Monetario: Cantidad de dinero gastada en un periodo específico de tiempo. Una alternativa también es considerar el monto promedio por compra para evitar colinealidad con la frecuencia.


Cada variable generalmente es evaluada en cinco niveles con base en quintiles (20% de los datos). En esta escala más es mejor, por ejemplo, entre menor es el tiempo desde la última compra, el valor en la escala es más alto.


Las tres variables son combinadas para crear una clasificación de tres dígitos. Por ejemplo, un cliente cuya última compra está en el 20% más reciente, que está en el 20% que más veces compra, y que está en el 20% que más ha comprado, tendría una clasificación 5-5-5.

Existen otras variantes del modelo RFM utilizando valores promedio, valores z, ponderando las variables, clasificando en dos etapas y otras alternativas más. Sin embargo, de momento no se abordarán estas variantes.

Para aplicar el análisis RFM las empresas deben tener un identificador del cliente y almacenar información de fecha y monto de cada transacción. Como ventajas del modelo RFM están su costo-beneficio, facilidad en su procedimiento y comprensión, estimación de la rentabilidad de los clientes y segmentos, y así su utilidad para el establecimiento de estrategias. Como desventajas, es un modelo principalmente descriptivo, tiene falta de precisión y presenta variabilidad entre empresas.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('data/b16_rfm.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Cliente  50000 non-null  object        
 1   Fecha    50000 non-null  datetime64[ns]
 2   Importe  50000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.1+ MB


In [4]:
# Calcular recencia (recency)
ultima_fecha = max(df['Fecha'])
recencia_df = df.groupby('Cliente')['Fecha'].max().reset_index()
recencia_df['recencia'] = (ultima_fecha - recencia_df['Fecha']).dt.days
recencia_df.tail(3)

,Cliente,Fecha,recencia
4997,CS6109,2022-10-30,500
4998,CS6110,2024-02-29,13
4999,CS6111,2023-05-24,294


In [5]:
# Calcular frecuencia (frequency)
frecuencia_df = df.groupby('Cliente').size().reset_index(name='frecuencia')
frecuencia_df.tail(3)

,Cliente,frecuencia
4997,CS6109,12
4998,CS6110,10
4999,CS6111,6


In [6]:
# Calcular monto (monetary)
monto_df= df.groupby('Cliente')['Importe'].sum().reset_index(name='monto')
monto_df.tail(3)

,Cliente,monto
4997,CS6109,16500
4998,CS6110,13980
4999,CS6111,7900


In [7]:
# Obtener quintiles
recencia_df['R'] = pd.qcut(recencia_df['recencia'], 5, labels=["5", "4", "3", "2", "1"])
frecuencia_df['F'] = pd.qcut(frecuencia_df['frecuencia'], 5, labels=["1", "2", "3", "4", "5"])
monto_df['M'] = pd.qcut(monto_df['monto'], 5, labels=["1", "2", "3", "4", "5"])

In [8]:
# Calcular puntaje RFM
rfm = pd.merge(recencia_df[['Cliente', 'R']], frecuencia_df[['Cliente', 'F']], on='Cliente')
rfm = pd.merge(rfm, monto_df[['Cliente', 'M']], on='Cliente')

In [13]:
rfm.head()

,Cliente,R,F,M,RF_SCORE,segment
0,CS1112,4,1,1,41,Prometedores
1,CS1113,4,3,4,43,Potencialmente leales
2,CS1114,4,2,3,42,Potencialmente leales
3,CS1115,1,4,5,14,En riesgo
4,CS1116,2,1,1,21,Hibernando


In [9]:
rfm["RF_SCORE"] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm.tail()

,Cliente,R,F,M,RF_SCORE
4995,CS6107,4,2,3,42
4996,CS6108,3,3,3,33
4997,CS6109,1,4,4,14
4998,CS6110,5,3,3,53
4999,CS6111,1,1,1,11


In [10]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Cliente   5000 non-null   object  
 1   R         5000 non-null   category
 2   F         5000 non-null   category
 3   M         5000 non-null   category
 4   RF_SCORE  5000 non-null   object  
dtypes: category(3), object(2)
memory usage: 93.5+ KB


In [11]:
seg_map = {
    r'[1-2][1-2]': 'Hibernando',
    r'[1-2][3-4]': 'En riesgo',
    r'[1-2]5': 'No se debe perder',
    r'3[1-2]': 'Cerca de suspender',
    r'33': 'Necesita atención',
    r'[3-4][4-5]': 'Clientes leales',
    r'41': 'Prometedores',
    r'51': 'Nuevos clientes',
    r'[4-5][2-3]': 'Potencialmente leales',
    r'5[4-5]': 'Campeones'}
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
rfm.tail()

,Cliente,R,F,M,RF_SCORE,segment
4995,CS6107,4,2,3,42,Potencialmente leales
4996,CS6108,3,3,3,33,Necesita atención
4997,CS6109,1,4,4,14,En riesgo
4998,CS6110,5,3,3,53,Potencialmente leales
4999,CS6111,1,1,1,11,Hibernando


In [14]:
rfm

,Cliente,R,F,M,RF_SCORE,segment
0,CS1112,4,1,1,41,Prometedores
1,CS1113,4,3,4,43,Potencialmente leales
2,CS1114,4,2,3,42,Potencialmente leales
3,CS1115,1,4,5,14,En riesgo
4,CS1116,2,1,1,21,Hibernando
...,...,...,...,...,...,...
4995,CS6107,4,2,3,42,Potencialmente leales
4996,CS6108,3,3,3,33,Necesita atención
4997,CS6109,1,4,4,14,En riesgo
4998,CS6110,5,3,3,53,Potencialmente leales
